# Simulating Language 5, Simple Innate Signalling (walkthrough)

This is a line-by-line walkthrough of the code for lab on simple signalling.

### Data Structures: a signalling matrix represented as a list of lists

A production system can be thought of as a matrix which maps meanings to signals. We are representing this as a list. Each member of the list is itself a list containing the association strengths for *one particular meaning*. Look at the example below:

```python
psys = [[1, 0, 0], [1, 2, 1], [3, 4, 4]]
```

Here, a production system called `psys` is defined: it has three members, representing three meanings. The length of the system `len(psys)` is equivalent to the number of meanings in the system. `psys[0]` is `[1, 0, 0]`, which are the association strengths for the first meaning (remember python indexes start from 0!). Each of these sub-lists has three members, representing three possible signals. So `psys[0][0]` is the strength of association between the first meaning and the first signal. We sometimes refer to these association strengths as "weights".

We can do the same thing to model a reception system, but in this case we are dealing with a system which maps from signals to meanings: so, if `rsys` is a reception system then each member of `rsys` is itself a list that contains the association strengths between a signal and several meanings.

 - Create a variable containing the following production matrix:

|    | s1 | s2 | s3 |
|----|----|----|----|
| m1 | 1  | 0  | 2  |
| m2 | 2  | 2  | 0  |
| m3 | 0  | 1  | 3  |


 - Print the weights for meaning m1

 - Print the weight for the connection between meaning m2 and signal s3

 - Create a variable containing the following reception matrix:


|    | m1 | m2 | m3 |
|----|----|----|----|
| s1 | 1  | 2  | 0  |
| s2 | 0  | 2  | 1  |
| s3 | 2  | 0  | 3  |

 - Print the weights for signal s3

 - Print the weight of the connection between signal s1 and meaning m2

## The code proper

The code begins by importing various random number and plotting modules:

In [1]:
import random

%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf')

### Function wta

The function `wta` ("winner takes all") takes a list of numbers (`items`) as its argument. This represents a row of a production or reception matrix. The function returns the index of the largest number in the list `items`. If there are multiple equally large numbers, then one of them is chosen at random.

In [2]:
def wta(items):
    maxweight = max(items)
    candidates = []
    for i in range(len(items)):
        if items[i] == maxweight:
            candidates.append(i)
    return random.choice(candidates)

`maxweight == max(items)` uses the built-in function `max` to calculate the maximum value of `items` and allocates this value to the variable `maxweight`.

`candidates = []` creates an empty list.

`for i in range(len(items)):` lets us look at each item in the list in turn, keeping track of where it is in the list. `range(len(items))` creates a sequence of numbers from 0 up to (but not including) the length of the list `items`. These represent each possible index of `items`, and in the the for loop, we go through each of these in turn, allocating it to the variable, `i`, and then carrying out everything in the next code block for each value of `i`:

```python
    if items[i] == maxweight:
        candidates.append(i)
```

This block of code checks each member of `items` in turn; if its value is equal to `maxweight`, then the index `i` is appended to (added to) the list of `candidates`.

After this loop has been completed, `candidates` will contain the indices of all the largest numbers.

`return random.choice(candidates)` returns a random choice from the numbers in the list, `candidates`. If there is only one number in `candidates`, then this is returned.

- Using the `wta` function and the variables you created above to store the production and reception matrices:

    - find the preferred signal for each meaning in turn
    
    - find the preferred meaning for each signal in turn
    
For example, if you called your production system `my_psys`, you could find the preferred signal for meaning 1 like this:

```python
wta(my_psys[0])
```

This takes the first row of the production system we defined earlier (`my_psys[0]`), then uses `wta` to find the index of the preferred signal for that row. Note that the `wta` function will only work if you pressed SHIFT+ENTER on the cell in the notebook above, otherwise the computer doesn't know what `wta` means.

### Function communicate

The function `communicate` plays a communication episode; it takes three arguments:
 - `speaker_system`, the production matrix of the speaker;
 - `hearer_system`, the reception matrix of the hearer; and
 - `meaning`, the index of the meanign which is to be communicated.
 

In [3]:
def communicate(speaker_system, hearer_system, meaning):
    speaker_signal = wta(speaker_system[meaning])
    hearer_meaning = wta(hearer_system[speaker_signal])
    if meaning == hearer_meaning:
        return 1
    else:
        return 0

In a communication episode, the speaker chooses a signal it uses to communicate `meaning`, and expresses this signal to the hearer; the hearer then chooses the meaning it understands by the speaker's signal. If the hearer's meaning is the same as the speaker's meaning, then the communication episode succeeds, otherwise it fails.

`speaker_signal = wta(speaker_system[meaning])` uses `speaker_system[meaning]` to extract a list of association strengths from the speaker's production matrix (`speaker_system`) for `meaning`, and then uses `wta` (see above) to find the index corresponding to the largest of these weights. This value is then stored in the variable `speaker_signal`.

`hearer_meaning = wta(hearer_system[speaker_signal])` uses `hearer_system[speaker_signal]` to extract a list of association strengths from the hearer's reception matrix (`hearer_system`) for `speaker_signal`, and then uses `wta` (see above) to find the index corresponding to the largest of these weights. This value is then stored in the variable `hearer_meaning`.

```python
    if meaning == hearer_meaning:
        return 1
    else:
        return 0
```

If the hearer's interpretation of the speaker's signal (`hearer_meaning`) equals the original value of `meaning` (i.e. the meaning the speaker was trying to convey) and thus the communication episode succeeds, then the function returns 1 (indicating success), otherwise (`else`) it returns 0 (indicating failure).

- Using the same matrices you created earlier, find out which of the meanings can be successfully communicated using these production and reception matrices.

### Function ca_monte

The function `ca_monte` (standing for "Communicative Accuracy Monte Carlo") is the main function in this program. It performs a Monte Carlo simulation, which runs a set number of communication episodes between a production system and a reception system, calculates how many of them were communicatively successful, and returns a trial-by-trial list of results. It takes three arguments:

- `speaker_system`, the production matrix of the speaker;
- `hearer_system`, the reception matrix of the hearer; and
- `trials`, the number of trials of the simulation, or the number of communicative episodes over which communicative accuracy should be calculated.


In [7]:
def ca_monte(speaker_system, hearer_system, trials):
    total = 0.
    accumulator = []
    for n in range(trials):
        total += communicate(speaker_system, hearer_system,
                            random.randrange(len(speaker_system)))
        accumulator.append(total / (n + 1))
    return accumulator

`total = 0.` creates a variable called total, which will store the number of successful communicative episodes. We use `0.` rather than `0` as a shorthand for `0.0`, which indicates that the eventual result isn't going to be a round number. In fact, this isn't strictly necessary for the version of Python we're using, but you're likely to see something like this in a lot of code you read.

`accumulator = []` creates a variable called `accumulator`, which will be used to build up a list of trial-by-trial success rates. We initialise the accumulator with an empty list: before we have conducted any trials, we don't have any results for success or failure.

`for n in range(trials):` sets up a loop to allow us to test communicative accuracy over and over again. `range(trials)` creates a sequence of numbers from 0 up to (but not including) `trials`, which is then traversed by the for loop.

`total += communicate(speaker_system, hearer_system, random.randrange(len(speaker_system)))` updates a running total of the number of communicative episodes that were successful. On each communicative episode, we choose a random meaning (that's what `random.randrange(len(speaker_system))` does - the length of `speaker_system` is the number of rows in their production matrix, which is the same as the number of meanings). Then we use the function `communicate` to see whether the speaker can successfully communicate this meaning to the hearer (`hearer_system`). We add the value returned by `communicate` (i.e. 0 or 1) to the existing value in `total`, which therefore contains the number of successful communicative episodes.

`accumulator.append(total / (n + 1))` builds up a list of exposure-by-exposure proportions of communicative episodes so far which have been successful. `total / (n + 1)` gives the total number of events so far that have been successful (stored in `total`) divided by the number of times we've been round the loop at this point. Note that the number of trials conducted so far is `n + 1`, not just `n`, because of the way `range` works. The first trial has `n` equal to 0, the second 1 and so on, so we have to add 1 to this number to get the number of trials completed. We then use `append` to add this value to `accumulator`, which is our building list of trial-by-trial success proportions.

`return accumulator` simply returns this list. Note that this line of code is outside of the the for loop. `accumulator` is only returned one the loop has run the necessary number of trials.

- What is the overall communicative accuracy for the matrices you defined earlier?

- change the `ca_monte` function so that the trailing decimal point is removed from the definition and run it again. What happens?

- create another matrix (maybe with more meanings and/or signals). What is its communicative accuracy?